In [1]:
import requests,datetime, time
from tqdm import tqdm
import pandas as pd
import logging
from secrets import WEATHER_API_KEY
logging.basicConfig(filename='weather.log', level=logging.INFO)

In [2]:
def process_response(airport_code,response):
    lines = response.split("\n")
    lines.pop(0)
    non_empty_lines = [line for line in lines if line.strip() != ""]
    map_obj = map(lambda s: airport_code+","+s, non_empty_lines)
    lines = list(map_obj)
    return "\n".join(lines)

weather_endpoint='https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?&aggregateHours=1&startDateTime={start_datetime}&endDateTime={end_datetime}&unitGroup=uk&contentType=csv&location={lat:.2f},{long:.2f}&US&key={apikey}'
apikey = WEATHER_API_KEY
def get_weather(airport_code, lattitude, longitude):
    response = requests.get(weather_endpoint.format(start_datetime="2015-01-01",end_datetime="2016-01-01",lat=lattitude, long=longitude, apikey=apikey)).text
    with open('./results/backups/{code}_weather_backup.csv'.format(code=airport_code),'w') as fd:
        fd.write(response)
    
    try:
        processed_weather = process_response(airport_code,response) #response.split("\n",1)[1]
        with open('./results/weather_airport.csv','a') as fd:
            fd.write(processed_weather)
        logging.info(airport_code+" - corect save")
    except Exception as exc:
        logging.error(exc)
        logging.info("Wrong data for airport: "+airport_code+" response: "+response)        

In [3]:
airports = pd.read_csv('data/airports.csv')
airports.loc[airports['IATA_CODE'] == 'ECP', ['LATITUDE']] = 30.35521
airports.loc[airports['IATA_CODE'] == 'ECP', ['LONGITUDE']] = -85.79956
airports.loc[airports['IATA_CODE'] == 'PBG', ['LATITUDE']] = 44.65349
airports.loc[airports['IATA_CODE'] == 'PBG', ['LONGITUDE']] = -73.46792
airports.loc[airports['IATA_CODE'] == 'UST', ['LATITUDE']] = 29.95518
airports.loc[airports['IATA_CODE'] == 'UST', ['LONGITUDE']] = -81.34300
airports.loc[[96,234,313],:]

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
96,ECP,Northwest Florida Beaches International Airport,Panama City,FL,USA,30.35521,-85.79956
234,PBG,Plattsburgh International Airport,Plattsburgh,NY,USA,44.65349,-73.46792
313,UST,Northeast Florida Regional Airport (St. August...,St. Augustine,FL,USA,29.95518,-81.34300


In [4]:
with open('./results/weather_airport.csv','w') as fd:
    fd.write("airport_code,Address,Date time,Minimum Temperature,Maximum Temperature,Temperature,Dew Point,Relative Humidity,Heat Index,Wind Speed,Wind Gust,Wind Direction,Wind Chill,Precipitation,Precipitation Cover,Snow Depth,Visibility,Cloud Cover,Sea Level Pressure,Weather Type,Latitude,Longitude,Resolved Address,Name,Info,Conditions\n")

In [5]:
for _, location in tqdm(airports.iterrows(), total=len(airports)):
    get_weather(location['IATA_CODE'],location['LATITUDE'],location['LONGITUDE'])

100%|██████████| 322/322 [11:23<00:00,  2.12s/it]
